In [4]:
! pip3 install elasticsearch --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd

In [9]:
# create documents for elasticsearch
df = pd.read_csv("../data/metadata_with_episode_dates.csv")

In [25]:
from elasticsearch import Elasticsearch, helpers
INDEX_NAME = "search-spotidy-dataset"

client = Elasticsearch(
  "https://527db13b131f4f1b90590b6f6c0039f6.us-central1.gcp.cloud.es.io:443",
  api_key="NkpsY1BJd0JLOE9tUzN1cnNqako6X2hNTzlhLUdTelNkbHdxTHlsNFlUdw=="
)


In [36]:
# deletes all records in elasticsearch
client.delete_by_query(index=[INDEX_NAME], body={"query": {"match_all": {}}})

ObjectApiResponse({'took': 33, 'timed_out': False, 'total': 435, 'deleted': 435, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [22]:
df = df.fillna("")

In [23]:
df

,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,release_date
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,2019-12-18
1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA,2020-01-30
2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,['en'],https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,2019-01-18
3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,['en-US'],https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,2019-03-01
4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,['en'],https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105355,spotify:show:416U8ZhubKrFHq8ynOaxfH,The Top 10,"Each week, John Rocha and Matt Knost breakdown...",The Top 10,['en-US'],http://thetop10.podomatic.com/rss2.xml,spotify:episode:7zzQnjBXqDApvnm1hLPzVY,The Top 10 - Re-List - Steve Martin Moves,Thanks to our patreon members for their suppor...,51.025850,show_416U8ZhubKrFHq8ynOaxfH,7zzQnjBXqDApvnm1hLPzVY,2019-03-29
105356,spotify:show:5rgmBAzsJ5znpV2b4WNDsb,Let's Grab Coffee Podcast,"After connecting with someone, what's the next...",George Khalife,['en-US'],https://anchor.fm/s/9043d60/podcast/rss,spotify:episode:7zzRRsjuymax0YSczpi0SU,Let's Grab Coffee E45 with Ross Paquette | Gro...,Ross founded Maropost in 2011 as a customer-ce...,33.364750,show_5rgmBAzsJ5znpV2b4WNDsb,7zzRRsjuymax0YSczpi0SU,2019-05-04
105357,spotify:show:56CjYLQWyMx1MkOEQmlubi,Coach Corey Wayne,Life & Peak Performance Coach. I Teach Self-Re...,Coach Corey Wayne,['en'],https://anchor.fm/s/4dd625c/podcast/rss,spotify:episode:7zzZJGsL8fwDOrduUkX91D,Maybe She Is Just Testing Me?,How to know if your woman is maybe just testin...,11.799950,show_56CjYLQWyMx1MkOEQmlubi,7zzZJGsL8fwDOrduUkX91D,2019-01-08
105358,spotify:show:7uddSH8MhaK3Q6YFlllbVZ,The Cricket Podcast,The best & funniest independent cricket podcas...,The Cricket Podcast,['en'],https://anchor.fm/s/9d3dcf0/podcast/rss,spotify:episode:7zzoT4r0Rhffyegk2HJ9N8,Ep 16: England In Danger,"In Episode 16, the boys evaluate England's per...",69.215350,show_7uddSH8MhaK3Q6YFlllbVZ,7zzoT4r0Rhffyegk2HJ9N8,2019-06-25


In [34]:
# prepare bulk data

actions = [
    {
        "_op_type": "index", # change to "update" when updating existing documents
        "_index": INDEX_NAME,
        "_id": row['episode_uri'],
        "doc": row.drop('episode_uri').drop('release_date').to_dict()
    }
    for index, row in df.iterrows()
]

In [37]:
# bulk operation on the documents
from elasticsearch.helpers import BulkIndexError

try:
    helpers.bulk(client, actions)
except BulkIndexError as e:
    print("An error occurred during bulk indexing:")
    print(e)

    # Optionally, print more detailed information about each error
    for error in e.errors:
        print(error)

In [39]:
client.search(index=INDEX_NAME, q="test")

ObjectApiResponse({'took': 63, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1857, 'relation': 'eq'}, 'max_score': 11.226463, 'hits': [{'_index': 'search-spotidy-dataset', '_id': 'spotify:episode:5vhEXNbx10zw7ojxJ2sz97', '_score': 11.226463, '_source': {'doc': {'show_uri': 'spotify:show:03lEaJfDP5ChTTjddAxSWX', 'show_name': 'First Grade Perspectives', 'show_description': 'So much learning happens in First Grade! Come learn with us. ', 'publisher': 'Testing app', 'language': "['en']", 'rss_link': 'https://anchor.fm/s/9b27d94/podcast/rss', 'episode_name': 'Inclusion', 'episode_description': 'First Graders discuss what they have learned about different abilities and how to best include others in their play.  ', 'duration': 13.544583333333335, 'show_filename_prefix': 'show_03lEaJfDP5ChTTjddAxSWX', 'episode_filename_prefix': '5vhEXNbx10zw7ojxJ2sz97'}}}, {'_index': 'search-spotidy-dataset', '_id': 'spotify:episode:6J02bts

In [47]:
query = {
    "query": {
        "query_string": {
            "query": "show_name:Kream in your Koffee"
        }
    }
}

response = client.search(index=INDEX_NAME, body=query)

In [48]:
response

ObjectApiResponse({'took': 75, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 18.550373, 'hits': [{'_index': 'search-spotidy-dataset', '_id': 'spotify:episode:000A9sRBYdVh66csG2qEdj', '_score': 18.550373, '_source': {'doc': {'show_uri': 'spotify:show:2NYtxEZyYelR6RMKmjfPLB', 'show_name': 'Kream in your Koffee', 'show_description': 'A 20-something blunt female takes on the world and gives you her take on it. Enjoy visits from special guests and friends to give insight and input into interesting situations.', 'publisher': 'Katie Houle', 'language': "['en']", 'rss_link': 'https://anchor.fm/s/11b84b68/podcast/rss', 'episode_name': '1: It’s Christmas Time!', 'episode_description': 'On the first ever episode of Kream in your Koffee, Katie talks about tips for Christmas shopping. We also get a little insight into who and what we’ll be hearing about in next weeks episode! ', 'duration'